In [6]:
from iohub.ngff import open_ome_zarr
from natsort import natsorted
from glob import glob
from pathlib import Path 
import torch
from torch.utils.data import Dataset


zarr_dir = "/mnt/efs/dlmbl/G-et/data/neuromast/Dataset/"
# defines input zarr file name with the zarr file structure
zarr_file = 'structured_celltype_classifier_data.zarr/*/*/*'
position_paths = natsorted(glob(zarr_dir + zarr_file))
# print(position_paths)


In [4]:
print("Number of positions: ", len(position_paths))

Number of positions:  1000


In [16]:
dataset = open_ome_zarr(position_paths[0], mode="r")
dataset.data.shape
all_chan = dataset.channel_names
chan = 'celltypes'
all_chan.index(chan)

2

In [20]:
class NeuromastDataset(Dataset):
    def __init__(self, file_path, class_channel: str = 'celltypes'):
        self.file_path = file_path
        #get the class index
        dataset = open_ome_zarr(self.file_path[0], mode="r")
        all_chan = dataset.channel_names
        self.c = all_chan.index(class_channel)
        

    def __len__(self):
        return len(self.file_path)

    def __getitem__(self, idx):
        dataset = open_ome_zarr(position_paths[idx], mode="r")
        image = dataset.data[:,0:self.c,:,:,:]
        label = dataset.data[:,self.c:self.c+1,:,:,:]
        return image, label

        


In [21]:
neuromast = NeuromastDataset(position_paths, class_channel='celltypes')
print(neuromast[0][0].shape)
print(neuromast[0][1].shape)

(1, 3, 73, 1024, 1024)
(1, 1, 73, 1024, 1024)
